In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
# Read Summarisation Model results from json file.

with open("../data/Model_Results/summarisation_results.jsonl", 'r') as json_file:
    json_list = list(json_file)

    
summarisationResults = {}
for json_str in json_list:
    
    result = json.loads(json_str)
    summaryList = sorted(result["sentence_scores"], key=lambda x:x[1], reverse=True)[:4]
    summary = ""
    for sentence in summaryList:
        summary += sentence[0] + ' '
    
    summarisationResults[result["id"]] = summary

In [3]:
summarisationData = pd.DataFrame(data=summarisationResults, index=[0])

In [4]:
display(summarisationData)

,11972.json,11685.json,11096.json,5209.json,9524.json,5962.json,7070.json,1046.json,12849.json,13270.json,...,11576.json,3461.json,9464.json,10227.json,11707.json,3425.json,2977.json,294.json,3580.json,8384.json
0,"""Building a wall"" on the border ""will take lit...","Those numbers show that as of October 2015, th...","If he did, he would have known that Senator Mc...","""…She supports taking $500 billion away from M...","Scott Walker helped run a ""criminal scheme"" to...",The campaign accurately quoted a figure whose ...,"So $30-31 million per year would, in fact, be ...",The Obama administration has emphasized many o...,"""It matters who’s leading the country, and it ...","Pence described the donors as major. ""The nati...",...,We’re not sure Sanders made that entirely clea...,"The $20 million designated for Cuba ""focuses o...",There also are no Asian or Pacific Islander Re...,"""The United States is in the longest stretch o...","""Secretary Clinton changes her position on thi...","Under the header ""New jobs created by the Stre...","They said low pay, increased work demands and ...","• Comstock, an adviser and frequent spokeswoma...","""House Republicans under Paul Ryan's leadershi...","""I will work in a bipartisan way to get it don..."


In [5]:
# Read DeClarE results from json file.
with open('../data/Model_Results/DeClarE_Results.json') as file:
    declareResults = json.load(file)

In [6]:
exist = 0
nExist = 0
for json_id in summarisationData.columns:

    if json_id not in declareResults.keys():
        del summarisationData[json_id]

In [7]:
pd.set_option('display.max_columns', None)
display(summarisationData)

,11096.json,9198.json,12996.json,13231.json,9092.json,6849.json,266.json,2966.json,8372.json,7823.json,12541.json,13334.json,6083.json,5913.json,6579.json,12456.json,4495.json,13125.json,110.json,7272.json,11867.json,9507.json,8434.json,8159.json,11778.json,3709.json,5732.json,1025.json,13102.json,12167.json,10540.json,4148.json,8705.json,10683.json,7057.json,9727.json,1926.json,11418.json,3652.json,8187.json,13077.json,12981.json,6697.json,10263.json,4264.json,6359.json,13142.json,9492.json,162.json,8019.json,13241.json,3782.json,11536.json,11828.json,10142.json,1266.json,6969.json,8733.json,2347.json,8050.json,1800.json,11690.json,11772.json,12779.json,3197.json,8489.json,2032.json,3195.json,13293.json,7303.json,5852.json,1659.json,12356.json,5388.json,8308.json,7217.json,12581.json,8405.json,11196.json,13223.json,4687.json,8210.json,2925.json,12265.json,10663.json,6449.json,8998.json,11513.json,10879.json,173.json,6517.json,88.json,10141.json,67.json,4589.json,10000.json,11363.json,11254.json,6379.json,11810.json,5031.json,5282.json,11249.json,9652.json,4373.json,7942.json,13247.json,13313.json,3255.json,9876.json,2328.json,10200.json,11744.json,7788.json,12822.json,5903.json,11713.json,1044.json,10402.json,10844.json,6162.json,5256.json,13369.json,13464.json,9547.json,12697.json,12499.json,4928.json,5750.json,7798.json,4627.json,12457.json,13039.json,10645.json,7960.json,1149.json,8473.json,2828.json,4700.json,2639.json,828.json,6411.json,361.json,11576.json,3461.json,10227.json,11707.json,3425.json,2977.json,294.json,3580.json
0,"If he did, he would have known that Senator Mc...","In her speech on April 12, 2014, Blackburn -- ...","It could be 30 million,"" Trump said. It could ...","At this point, Clinton was secretary of state....","""It is important to note that the ‘300,000’ wa...",Conservative health experts say Romney’s refor...,"""You said you would vote against the Patriot A...",The answer depends on the rhetorical flourish ...,"""... because the bill only authorizes the impo...","""Barbara Buono has relied exclusively on negat...","In Trump’s case, that means the rules committe...","For that fact check, we also confirmed that wh...","But here’s the position Citizens is in now,"" S...","Broadly, Tom Perkins, city attorney for the Ci...","""And what did President Obama and his National...","""It's entirely possible,"" said Toomey ""that th...",What about comparing only the first 968 days f...,"""I think it’s real,"" she said during a Sept. 2...","Thompson added, ""The Iraq Study Group reported...","""Nobody can claim Social Security isn’t part o...","According to CNN, what Cruz said ""is categoric...","""That's the hottest year on record in the Unit...","Under the photo, Mark Obenshain, the Republica...","The fact sheet says, ""Each year, approximately...","Trump said, ""China, they don't like to tell us...","He said he had not smoked marijuana, and his b...","""What president has the worst record on female...",The economic stimulus bill enacted in February...,"We wanted to look at ""the treaty cutting Russi...","This, combined with the fact that state fundin...","""When did the decline of coal start? ""That sta...",Sanders ended the tweet with his signature has...,The law added about3.1 million young adults to...,"• ""63 percent of the American people chose not...",That would be speculation -- but that speculat...,"""The president's come out with rules that say ...",And in a July 2009 speech at the Council on Fo...,"""African-American youth unemployment is 51 per...",He took exactly none of his own deficit reduct...,"""He’s the person who invented the scheme to re...",It usually goes something like this: Women ear...,About $2.4 billion of that amount was allocate...,"Home values collapsed, home construction jobs ...",That percentage increase placed Austin 10th na...,"""You know there was a poll, I guess it was abo...","Some of those conflicts, such as Afghanistan, ...","""About four months ago, I read on the front

In [8]:
# Read Justification data from tsv into dataframe
justificationData = pd.read_csv ('../data/Justification_Data/Justification_Data.tsv', sep='\t')


# Removed Unwanted Column
del justificationData["Unnamed: 0"]

display(justificationData)

,json_file_id,claim,justification,justification_label
0,3197.json,Twelve judges have thrown out legal challenges...,They dismissed the cases because of procedural...,distortion
1,8489.json,"Terry McAuliffe wants to raise your taxes $1,7...",Cuccinelli says McAuliffe wants to raise taxes...,emphasis
2,2032.json,The top 1 percent pay over half of the entire ...,The top one-fifth of the population -- what on...,distortion
3,3195.json,Says he never said he would keep education fun...,"Yes, it's a small cut from a proposed budget o...",unfounded
4,13293.json,"Says Sen. Rob Portman said, It would be irresp...",Strickland said Portman went on television and...,distortion
...,...,...,...,...
169,5750.json,The bulk of the people who are shot with a wea...,"Biden said ""the bulk"" of people who are shot w...",distortion
170,10402.json,Says Nikita Khrushchev brought an extra shoe t...,Commentator Arthur Cyr stated that Nikita Khru...,emphasis
171,10844.json,Every president since Franklin Delano Roosevel...,"Ryan said, ""Every president since Franklin Del...",emphasis
172,6162.json,Obama has racked up more debt than any preside...,"Experts note, too, that the weak economy has c...",distortion


In [9]:
summarisationDataDict = {}

for json_id in summarisationData.columns:
    
    features = {}
    
    justDataRow = justificationData.loc[justificationData['json_file_id'] == json_id]
    
    if len(justDataRow["claim"].values) == 0:
        continue
    
    features["claim"] = justDataRow["claim"].values[0]
    features["justification_label"] = justDataRow["justification_label"].values[0]
    
    summarisationDataDict[json_id] = features

In [10]:
summarisationData = pd.DataFrame().from_dict(data=summarisationDataDict, orient="index")

summarisationData["justification_label"].value_counts()

distortion    63
unfounded     47
emphasis      41
Name: justification_label, dtype: int64

In [11]:
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [12]:
justification_classes = {"distortion": 22, "unfounded": 6, "emphasis": 0}
for just_class in justification_classes:
    
    summarisationDataIndices = list(summarisationData.index[summarisationData['justification_label'] == just_class])
    
    randIndexList = random.sample(summarisationDataIndices, justification_classes[just_class])
    if randIndexList:
        summarisationData.drop(randIndexList, inplace=True)

In [13]:
summarisationData["justification_label"].value_counts()

unfounded     41
emphasis      41
distortion    41
Name: justification_label, dtype: int64

In [14]:
tempData = summarisationData

y = tempData["justification_label"].to_frame()

In [15]:
# Create directories and train-eval-test datasets

for i in range(10):
    filePath = '../data/Train_Eval_Test_Data/Iteration' + str(i + 1)
    
    exists = os.path.exists(filePath)

    if not exists:
        os.makedirs(filePath, exist_ok=True)

In [16]:
# Split train-val-test dataset for 10-Fold validation

skf = StratifiedKFold(n_splits=10)
splitCounter = 0
for train_index, test_index in skf.split(tempData, tempData['justification_label'].to_frame()):

    X_trainTest = tempData.iloc[train_index]
    Y_trainTest = tempData.iloc[train_index]['justification_label']

    #   Split a part from train data with length equal to test to be used as validation
    X_train, X_val, y_train, y_val = train_test_split(X_trainTest, Y_trainTest,stratify=Y_trainTest, test_size=len(test_index))
    
    X_train.to_csv('../data/Train_Eval_Test_Data/Iteration' + str(splitCounter + 1) +'/train' + str(splitCounter + 1) + '.tsv', sep="\t")
    X_val.to_csv('../data/Train_Eval_Test_Data/Iteration' + str(splitCounter + 1) +'/val' + str(splitCounter + 1) + '.tsv', sep="\t")    
    tempData.iloc[test_index].to_csv('../data/Train_Eval_Test_Data/Iteration' + str(splitCounter + 1) + '/test' + str(splitCounter + 1) + '.tsv', sep="\t")

    splitCounter += 1

#### Find inter-annotator aggreement

In [17]:
from os import listdir
from os.path import isfile, join
from sklearn.metrics import accuracy_score, f1_score

In [18]:
# Read Forms_per_annotator from json file.
with open('../data/Annotation_Results/Forms_per_Annotator.json') as file:
    formsPerAnnotator = json.load(file)

In [19]:
resultsDir = "../data/Annotation_Results/"
forms = [join(resultsDir,file) for file in listdir(resultsDir) if isfile(join(resultsDir, file))]

numberOfAnnotators = len(formsPerAnnotator.keys())
meanAccuracy = 0.0
meanF1 = 0.0

for key in formsPerAnnotator.keys():
    print("------------------------------")
    print("Parsing ", key)
    for form in forms:
        
        if "Forms_per_Annotator" in form:
            forms.remove(form);
            continue
        
        file = open(form, 'r')
        firstLine = file.readline()
        
        firstLineInfo = firstLine.split('#')
        
        formId = firstLineInfo[1]
        formFileName = firstLineInfo[2].replace("\n",'')
        
        if formId not in formsPerAnnotator[key]:
            continue
        
        print("Selecting form: ",formId)
        resultDf = pd.DataFrame()
        
        jsonIds = []
        justificationLabels = []
        
        with open(join("../annotation/claim-justification_annotator/public/",formFileName), 'r') as formJson:
            formData = json.loads(formJson.read())
        
        formLines = file.readlines()
        for line in formLines[1:]:

            lineData = line.split(',')

            for i in range (0,len(lineData)):
                lineData[i] = lineData[i].replace('"', '').strip()
            
            for i in formData:
                if i['id'] == float(lineData[0]):
                    jsonId = i['json_file_id']
                    break
            
            if jsonId not in list(summarisationData.index):
                continue
            
            jsonIds.append(jsonId)
            if (lineData[1] == "true"):
                justificationLabels.append("distortion") 
            elif (lineData[2] == "true"):
                justificationLabels.append("emphasis")
            elif (lineData[3] == "true"):
                justificationLabels.append("unfounded")
            else:
                justificationLabels.append("unclear")
        
        forms.remove(form);
        
    resultDf["json_ids"] = jsonIds
    resultDf["jsutification_label"] = justificationLabels
    
    annotatorResults = list(resultDf["jsutification_label"])
    majorityResults  = list(summarisationData.loc[list(resultDf["json_ids"])]["justification_label"])
    
    accuracyScore = accuracy_score(majorityResults,annotatorResults)        
    f1Score = f1_score(majorityResults,annotatorResults, average="macro")
    
    print("Accuracy Score: ", round(accuracyScore,2))
    print("F1 Score: ", round(f1Score,2))

    meanAccuracy += accuracyScore
    meanF1 += f1Score

print("Annotator Mean Accuracy Score is: ", round(meanAccuracy/numberOfAnnotators,2))
print("Annotator Mean F1 Score is: ", round(meanF1/numberOfAnnotators,2))

------------------------------
Parsing  annotator1
Selecting form:  _vcike0dhl
Selecting form:  _4k5w7qyi4
Accuracy Score:  0.48
F1 Score:  0.38
------------------------------
Parsing  annotator2
Selecting form:  _emun2cjhs
Selecting form:  _pf4thc9pa
Accuracy Score:  0.67
F1 Score:  0.51
------------------------------
Parsing  annotator3
Selecting form:  _i5j0r2042
Selecting form:  _18grnudgo
Selecting form:  _s7g02jdld
Accuracy Score:  0.43
F1 Score:  0.39
------------------------------
Parsing  annotator4
Selecting form:  _q2huun4xq
Selecting form:  _bda6rbbri
Selecting form:  _nc9mw06hf
Selecting form:  _icl52gift
Accuracy Score:  0.65
F1 Score:  0.64
------------------------------
Parsing  annotator5
Selecting form:  _r1a14srpr
Selecting form:  _hlbqc9sgz
Selecting form:  _9tec3uggs
Accuracy Score:  0.25
F1 Score:  0.18
------------------------------
Parsing  annotator6
Selecting form:  _wjg24bjlj
Selecting form:  _149v2kjrz
Accuracy Score:  0.84
F1 Score:  0.82
------------------